<a href="https://colab.research.google.com/github/Spoorthi-2005/TitanicSurvivalprediction/blob/main/TitanicSurvival.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#import necessary libraries


In [45]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

2.Load Datset


In [46]:
data=pd.read_csv("/content/drive/MyDrive/Titanic-Dataset.csv")

In [47]:
print("Column names:", data.columns)

Column names: Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


In [48]:
print("First few rows of the dataset:")
print(data.head())

First few rows of the dataset:
   PassengerId  Survived  Pclass  ...     Fare Cabin  Embarked
0            1         0       3  ...   7.2500   NaN         S
1            2         1       1  ...  71.2833   C85         C
2            3         1       3  ...   7.9250   NaN         S
3            4         1       1  ...  53.1000  C123         S
4            5         0       3  ...   8.0500   NaN         S

[5 rows x 12 columns]


3.Data Preprocessing

3.1 Drop Unnecessary columns

In [19]:
data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

3.2 Handle Missing Values


In [49]:
data['Age'].fillna(data['Age'].median(), inplace=True)
data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)
data['Fare'].fillna(data['Fare'].median(), inplace=True)

<ipython-input-49-697b529a860d>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Age'].fillna(data['Age'].median(), inplace=True)
<ipython-input-49-697b529a860d>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', tr

In [50]:
data['FamilySize'] = data['SibSp'] + data['Parch'] + 1
data['IsAlone'] = np.where(data['FamilySize'] == 1, 1, 0)
data['Title'] = data['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
data['Title'] = data['Title'].replace(['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
data['Title'] = data['Title'].replace('Mlle', 'Miss')
data['Title'] = data['Title'].replace('Ms', 'Miss')
data['Title'] = data['Title'].replace('Mme', 'Mrs')
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
data['Title'] = data['Title'].map(title_mapping)
data['Title'] = data['Title'].fillna(0)

3.3 Encode Categorical Variables

In [51]:
label_encoder = LabelEncoder()
data['Sex'] = label_encoder.fit_transform(data['Sex'])
data['Embarked'] = label_encoder.fit_transform(data['Embarked'])

# Drop Name column as it is no longer needed
data.drop(['Name'], axis=1, inplace=True)

4.Feature and Target Separation

In [52]:
X=data.drop('Survived',axis=1)
Y=data['Survived']

5.Train-Test Split

In [53]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

In [29]:
non_numeric_columns = X_train.select_dtypes(include=['object', 'bool']).columns

# Check for any remaining non-numeric columns
if non_numeric_columns.size > 0:
    print("Non-numeric columns found:", non_numeric_columns)
    # Drop or encode non-numeric columns
    for col in non_numeric_columns:
        if col in X_train.columns:
            X_train.drop(col, axis=1, inplace=True)
            X_test.drop(col, axis=1, inplace=True)
            print(f"Dropped column: {col}")

Non-numeric columns found: Index(['Name', 'Ticket', 'Cabin', 'Embarked_Q', 'Embarked_S', 'Pclass_2',
       'Pclass_3'],
      dtype='object')
Dropped column: Name
Dropped column: Ticket
Dropped column: Cabin
Dropped column: Embarked_Q
Dropped column: Embarked_S
Dropped column: Pclass_2
Dropped column: Pclass_3


6.Normalize Numerical Data

In [30]:
print("Data types of each column:")
print(X_train.dtypes)

Data types of each column:
PassengerId    float64
Sex            float64
Age            float64
SibSp          float64
Parch          float64
Fare           float64
dtype: object


In [31]:
numerical_columns = X_train.select_dtypes(include=['int64', 'float64']).columns

In [32]:
scaler = StandardScaler()

In [33]:
X_train[numerical_columns] = scaler.fit_transform(X_train[numerical_columns])

Train the model

In [34]:
X_train = X_train.astype(float)
X_test = X_test.astype(float)

In [36]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train, Y_train)

RandomForestClassifier(random_state=42)

Make Predictions

In [37]:
Y_pred = model.predict(X_test)

#Evaluate model performance

In [38]:
accuracy = accuracy_score(Y_test, Y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.42


In [40]:
precision = precision_score(Y_test, Y_pred)
print(f"Precision: {precision:.2f}")


Precision: 0.36


In [41]:
recall = recall_score(Y_test, Y_pred)
print(f"Recall: {recall:.2f}")


Recall: 0.50


In [42]:
f1 = f1_score(Y_test, Y_pred)
print(f"F1 Score: {f1:.2f}")

F1 Score: 0.42


Classification Report


In [44]:
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.51      0.37      0.43       105
           1       0.36      0.50      0.42        74

    accuracy                           0.42       179
   macro avg       0.44      0.44      0.42       179
weighted avg       0.45      0.42      0.43       179



HYPER PARAMETER TUNING

In [60]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset
data = pd.read_csv("/content/drive/MyDrive/Titanic-Dataset.csv")

# Data preprocessing
# Drop columns that are not useful for the model
data.drop(['PassengerId', 'Ticket', 'Cabin'], axis=1, inplace=True)

# Handle missing values
data['Age'] = data['Age'].fillna(data['Age'].median())
data['Embarked'] = data['Embarked'].fillna(data['Embarked'].mode()[0])
data['Fare'] = data['Fare'].fillna(data['Fare'].median())

# Feature Engineering
data['FamilySize'] = data['SibSp'] + data['Parch'] + 1
data['IsAlone'] = np.where(data['FamilySize'] == 1, 1, 0)
data['Title'] = data['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
data['Title'] = data['Title'].replace(['Lady', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
data['Title'] = data['Title'].replace('Mlle', 'Miss')
data['Title'] = data['Title'].replace('Ms', 'Miss')
data['Title'] = data['Title'].replace('Mme', 'Mrs')
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
data['Title'] = data['Title'].map(title_mapping)
data['Title'] = data['Title'].fillna(0)

# Encode categorical variables
label_encoder = LabelEncoder()
data['Sex'] = label_encoder.fit_transform(data['Sex'])
data['Embarked'] = label_encoder.fit_transform(data['Embarked'])

# Drop Name column as it is no longer needed
data.drop(['Name'], axis=1, inplace=True)

# Separate features and target variable
X = data.drop('Survived', axis=1)
y = data['Survived']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize numerical data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Hyperparameter Tuning using GridSearchCV
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf']
}

model = SVC()
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Best parameters and best score
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation score: ", grid_search.best_score_)

# Train the best model
best_model = grid_search.best_estimator_

# Make predictions
y_pred = best_model.predict(X_test)

# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print(classification_report(y_test, y_pred))

Best parameters found:  {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
Best cross-validation score:  0.8300108342361863
Accuracy: 0.82
              precision    recall  f1-score   support

           0       0.83      0.87      0.85       105
           1       0.80      0.74      0.77        74

    accuracy                           0.82       179
   macro avg       0.81      0.80      0.81       179
weighted avg       0.81      0.82      0.81       179

